In [1]:
# Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy
import pandas as pd
import statistics
import time
from tqdm import tqdm
%matplotlib qt

In [2]:
xls = pd.ExcelFile('Data_2D_EXP.xlsx')
Reservoir_data = pd.read_excel(xls, 'Sheet1')
print(Reservoir_data)
print("\n")
phi=np.float64(Reservoir_data['Value'][0])
L=np.float64(Reservoir_data['Value'][1])
W=np.float64(Reservoir_data['Value'][1])
H=np.float64(Reservoir_data['Value'][2])
mu=np.float64(Reservoir_data['Value'][3])
Ct=np.float64(Reservoir_data['Value'][4])
Bo=np.float64(Reservoir_data['Value'][5])
Del_t=np.float64(Reservoir_data['Value'][6])
Nt=int(Reservoir_data['Value'][7])
N=int(Reservoir_data['Value'][8])
Del_x=L/N
print("\nSize of Block: ", Del_x, "ft")
Pi=float(Reservoir_data['Value'][9])
qsc=float(Reservoir_data['Value'][10])
k=float(Reservoir_data['Value'][11])
print("\nIC \n at t=0, Reservoir Pressure is ", Pi, " psi \nBCs \n Surface production rate at the centre block, i.e. (",(N+1)/2,",",(N+1)/2,") is ", qsc, "stb/day \n at the reservoir boundaries (NEUMANN BC), NO  FLOW is there i.e., (Del_P/Del_x)=0\n")
Psc=5.615*Bo*Del_t*qsc/H/(Del_x**2)/phi/Ct
eta=0.00633*k*Del_t/phi/mu/Ct/(Del_x**2)
print(eta)
print("\n")


# Estimation of Penta-Diagonal Matrix for Implicit formulation
etamat=np.zeros((N**2,N**2))
for i in range(0,N):
    for j in range(0,N):
        if j==i-1:
            etamat[i,j]=-eta
            
        elif j==i:
            etamat[i,j]=1+3*eta

        elif j==i+1:
            etamat[i,j]=-eta
            
etamat[0,0]=1+2*eta
etamat[N-1,N-1]=1+2*eta
        
etamat[(N**2)-N:N**2,(N**2)-N:N**2]=etamat[0:N,0:N]

for k in range(1,N-1):
    etamat[k*N:(k+1)*N,k*N:(k+1)*N]=etamat[0:N,0:N]+(eta*np.identity(N))
    
for k in range(1,N):
    etamat[k*N:(k+1)*N,(k-1)*N:k*N]=(-eta)*np.identity(N)
    etamat[(k-1)*N:k*N,k*N:(k+1)*N]=(-eta)*np.identity(N)

print("Penta-diagonal Matrix or eta-matrix is estimated as: \n"+str(etamat))
pendiag=pd.DataFrame(etamat)
etainv=np.linalg.inv(etamat)

P_=np.full((Nt+1,N**2),Pi,dtype=np.float64)
for i in range(1,Nt+1):
    B=P_[i-1,:]
    F=np.zeros((N**2))
    F[(18-1)*N+18-1]=-Psc
    F[(53-1)*N+53-1]=-Psc
    F[(18-1)*N+53-1]=-Psc
    F[(53-1)*N+18-1]=-Psc
    P_[i,:]=np.dot(etainv,B+F)

P=np.full((Nt+1,N,N),0,dtype=np.float64)
for i in range(Nt+1):
    P[i]=P_[i].reshape(N,N).transpose()

    
print("\n")
m=[]
for i in range(N):
    m.append((i+0.5)*Del_x)
plt.figure(figsize=(8,8))
k=plt.axes(projection='3d')
X, Y = np.meshgrid(m, m)
k.plot_wireframe(X, Y,P[math.floor(Nt/10),:,:],color='r')
k.plot_wireframe(X, Y,P[math.floor(Nt/2),:,:],color='y')
k.plot_wireframe(X, Y,P[10*math.floor(Nt/10),:,:],color='m')
k.set_xlabel('Reservoir Length in ft')
k.set_ylabel('Reservoir Width in ft')
k.set_zlabel('Pressure in psi')
print('Red => Time Step= '+str(math.floor(Nt/10)))
print('Yellow => Time Step= '+str(math.floor(Nt/2)))
print('Magenta => Time Step= '+str(10*math.floor(Nt/10)))
plt.grid()
plt.show()

                                             Property Annotation        Value
0                               Porosity in fraction:        phi     0.500000
1                             Reservoir Length in ft:          L  1000.000000
2                             Reservoir Height in ft:          H    10.000000
3                                    Viscosity in cp:         mu     0.800000
4                    Total Compressibility in psi^-1:         Ct     0.000005
5            Fluid Formation Volume Factor in rb/stb:         Bo     1.100000
6                                  Time Step in days:      Del_t     0.010000
7                               Number of Time Steps:         Nt   500.000000
8                Number of Blocks in both directions:          N    71.000000
9                   Reservoir Pressure in psi at t=0:         Pi  2000.000000
10  Surface flow rate at center of reservoir in st...        qsc   100.000000
11                                 Permeability in md          k

In [3]:
# Dynamic Heatmap of the pressure simulation with 20 iterations
%matplotlib qt
Pn=np.full(((Nt//25),N,N),0,dtype=np.float64)
for l in range(0,(Nt//25)):
    for i in range(0,N):
        for j in range(0,N):
            if l==0:
                Pn[l,i,j]=P[1,i,j]
            elif l==1:
                Pn[l,i,j]=P[5*l,i,j]
            else:
                Pn[l,i,j]=P[25*(l-1),i,j]


fig=plt.figure(figsize=(8,8))
plt.style.use('default')
for i in tqdm(range(0,(Nt//25))):
    time.sleep(1.25)
    plt.imshow(Pn[i])
    fig.canvas.draw()
    fig.canvas.flush_events()

100%|██████████| 20/20 [00:28<00:00,  1.44s/it]
